# Physics based pedestrian modeling

<p align="center">
    <a href="" alt="Repository">
	   <img src="https://img.shields.io/badge/GitHub-181717?style=flat&logo=github&logoColor=white" /></a>
    <a href="https://github.com/c-pouw/physics-based-pedestrian-modeling/actions/workflows/testing.yml" alt="Unit Tests">
       <img src="https://github.com/c-pouw/physics-based-pedestrian-modeling/actions/workflows/testing.yml/badge.svg" /></a>
	<a href="https://github.com/c-pouw/physics-based-pedestrian-modeling/actions/workflows/integration-tests.yaml" alt="Integration Tests">
       <img src="https://github.com/c-pouw/physics-based-pedestrian-modeling/actions/workflows/integration-tests.yaml/badge.svg" /></a>
	<a href="https://github.com/c-pouw/physics-based-pedestrian-modeling/actions/workflows/pages/pages-build-deployment" alt="pages-build-deployment">
	   <img src="https://github.com/c-pouw/physics-based-pedestrian-modeling/actions/workflows/pages/pages-build-deployment/badge.svg" /></a>
    <a href="https://pypi.python.org/pypi/physics-based-pedestrian-modeling" alt="pypi version">
       <img src="https://img.shields.io/pypi/v/physics-based-pedestrian-modeling.svg" /></a>
    <a href="#">
       <img src="https://img.shields.io/pypi/pyversions/physics-based-pedestrian-modeling" alt="PyPI - Python Version" /></a>
    <a href="https://github.com/psf/black">
       <img src="https://img.shields.io/badge/code%20style-black-000000.svg" alt="Code style: black" /></a>
	<a href="https://colab.research.google.com/github/c-pouw/physics-based-pedestrian-modeling/blob/master/usage_notebooks/physped_quick_start.ipynb">
	   <img src="https://colab.research.google.com/assets/colab-badge.svg"></a>
	<a href="https://arxiv.org/abs/2407.20794">
	   <img src="https://img.shields.io/badge/arXiv-2407.20794-b31b1b.svg?style=flat" alt="arXiv" /></a>
</p>

# Project Overview

Python package to create physics-based pedestrian models from pedestrian trajectory measurements. This package is an implementation of the data-driven generalized pedestrian model presented in:

Pouw, C. A. S., van der Vleuten, G., Corbetta, A., & Toschi, F. (2024). Data-driven physics-based modeling of pedestrian dynamics. Preprint, https://arxiv.org/abs/2407.20794


## Documentation

* Documentation: https://c-pouw.github.io/physics-based-pedestrian-modeling.

### This notebook shows how to use the generalized pedestrian model with one of the public data sets.

#### First install the physics-based-pedestrian-modeling package
Note: You might need to restart the session after installing the package.

In [ ]:
!pip install --upgrade physics-based-pedestrian-modeling

#### Importing libraries

In [ ]:
import logging
from pprint import pformat

from omegaconf import OmegaConf

from physped.utils.config_utils import initialize_hydra_config, set_plot_style
from physped.io.readers import trajectory_reader
from physped.preprocessing.trajectories import preprocess_trajectories, process_slow_modes
from physped.core.functions_to_discretize_grid import learn_potential_from_trajectories
from physped.core.trajectory_simulator import simulate_trajectories
from physped.visualization.plot_trajectories import plot_trajectories

#### Configuration and setup

Steps:
- Read the configuration file
- Setup the logging
- Set the matplotlib plot tyle
- Change the working directory

Available configuration files:
- Narrow corridor paths
- Intersecting paths
- Train station platform paths

In [ ]:
# Select one of the available environments
environments = {
    'Narrow corridor paths': 'single_paths',
    'Intersecting paths': 'intersecting_paths',
    'Train station platform paths': 'asdz_pf12'
}
env_name = environments.get('Narrow corridor paths')

# Initialize Hydra configuration
config = initialize_hydra_config(env_name)

# set logging level
log = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, force = True)

# set plot style
set_plot_style(use_latex=False)

The default model parameters are set to:
- Noise intensiy $\sigma = 0.9\, \textrm{m s}^{-3/2}$
- Relaxation time $\tau_x = \tau_u = 0.5$ s
- Integration timestep $\Delta t=\frac{1}{f}$

In [ ]:
logging.info(
    "MODELING PARAMETERS: \n" + 
    pformat(OmegaConf.to_container(config.params.model, resolve=True), depth=1)
    )

#### Optional: overwriting the model parameters
Any parameter in the config can be overwritten like demonstrated in the cell below

In [ ]:
overwrite_config_params = False
if overwrite_config_params:
    config.params.model.sigma = 0.9
    config.params.model.taux = 0.5
    config.params.model.tauu = 0.5
    config.params.model.dt = 1/config.params.model.fps

#### Read the raw trajectories 
In this notebook the public trajectory data set is read from a remote repository on 4TU. Be aware that this might throw an error when the remote repository is not accessible.

In [ ]:
trajectories = trajectory_reader[env_name](config)
logging.info('\n' + pformat(trajectories.head()))

#### Preprocess the trajectories.
Steps:
- Rename the columns
- Prune short trajectories
- Add a trajectory index
- Compute velocities in Cartesian coordinates
- Convert velocities to polar coordinates

In [ ]:
trajectories = preprocess_trajectories(trajectories, config=config)

#### Compute the slow dynamics.
Steps:
- Compute the slow velocities $\vec{u}_s$
- Compute the slow positions $\vec{x}_s$

In [ ]:
trajectories = process_slow_modes(trajectories, config)

#### Plot the preprocessed trajectories

In [ ]:
plot_trajectories(trajectories, config, "recorded")

#### Compute the piecewise potential
Steps:
- Create the lattice to discretize the slow dynamics
- Fit the actual dynamics conditioned to the slow dynamics with Gaussian distributions

In [ ]:
reduce_lattice_size = True
if reduce_lattice_size:
    config.params.grid.r.list = [0, 0.5, 1, 1.5, 2, 2.5]
    config.params.grid.spatial_cell_size = 0.4
piecewise_potential = learn_potential_from_trajectories(trajectories, config)

#### Simulate new trajectories with the model
- Sample initial conditions from the trajectory measurements
- Integrate the generalized pedestrian model

The number of trajectories to simulate can be overwritten in the configuration as shown below. 

In [ ]:
config.params.simulation.ntrajs = 25
simulated_trajectories = simulate_trajectories(piecewise_potential, config, trajectories)

#### Plot the simulated trajectories

In [ ]:
plot_trajectories(simulated_trajectories, config, "simulated")